In [1]:
import collections
import networkx as nx
import random
import pickle
import cvxpy as cp
import numpy as np

def decomposition(graph, size):
    node_degree = [[i, 0] for i in range(size)]
    node_to_node = [[] for i in range(size)]
    node_degree_dict = collections.defaultdict(int)
    node_set = set()
    for edge in graph:
        node1, node2 = edge[0], edge[1]
        node_degree[node1][1] += 1
        node_degree[node2][1] += 1
        if node1 in node_to_node[node2] or node2 in node_to_node[node1]:
            print("Invalid input graph! Double edge! ("+str(node1) +", "+ str(node2)+")")
            exit()
        if node1 == node2:
            print("Invalid input graph! Circle! ("+str(node1) +", "+ str(node2)+")")
            exit()

        node_to_node[node1].append(node2)
        node_to_node[node2].append(node1)
        node_degree_dict[node1] += 1
        node_degree_dict[node2] += 1
        node_set.add(node1)
        node_set.add(node2)

    node_degree = sorted(node_degree, key = lambda x: x[1])
    node_degree[:] = node_degree[::-1]
    subgraphs = []
    min_num = node_degree[0][1]
    while node_set:
        subgraph = []
        for i in range(size):
            node1, node1_degree = node_degree[i]
            if node1 not in node_set:
                continue
            for j in range(i+1, size):
                node2, node2_degree = node_degree[j]
                if node2 in node_set and node2 in node_to_node[node1]:
                    subgraph.append((node1, node2))
                    node_degree[j][1] -= 1
                    node_degree[i][1] -= 1
                    node_degree_dict[node1] -= 1
                    node_degree_dict[node2] -= 1
                    node_to_node[node1].remove(node2)
                    node_to_node[node2].remove(node1)
                    node_set.remove(node1)
                    node_set.remove(node2)
                    break
        subgraphs.append(subgraph)
        for node in node_degree_dict:
            if node_degree_dict[node] > 0:
                node_set.add(node)
        node_degree = sorted(node_degree, key = lambda x: x[1])
        node_degree[:] = node_degree[::-1]
    return subgraphs

def getSubGraphs(graph, size):
    subgraphs = list()
    for i in range(80-1):
        M1 = nx.max_weight_matching(graph)
    if nx.is_perfect_matching(graph, M1):
        graph.remove_edges_from(list(M1))
        subgraphs.append(list(M1))
    else:
        edge_list = list(graph.edges)
        random.shuffle(edge_list)
        graph.remove_edges_from(edge_list)
        graph.add_edges_from(edge_list)

    # use greedy algorithm to decomposes the remaining part
    rpart = decomposition(list(graph.edges), size)
    for sgraph in rpart:
        subgraphs.append(sgraph)
    return subgraphs

        
def graphToLaplacian(subGraphs, size):
    L_matrices = list()
    for i, subgraph in enumerate(subGraphs):
        tmp_G = nx.Graph()
        tmp_G.add_edges_from(subgraph)
        tmp_G.add_nodes_from(range(size)) 
        L_matrices.append(nx.laplacian_matrix(tmp_G, nodelist=list(range(size))).todense())

    return L_matrices
    
def getProbability(L_matrices, commBudget):
    num_subgraphs = len(L_matrices)
    p = cp.Variable(num_subgraphs)
    L = p[0]*L_matrices[0]
    for i in range(num_subgraphs-1):
        L += p[i+1]*L_matrices[i+1]
    eig = cp.lambda_sum_smallest(L, 2)
    sum_p = p[0]
    for i in range(num_subgraphs-1):
        sum_p += p[i+1]

    # cvx optimization for activation probabilities
    obj_fn = eig
    constraint = [sum_p <= num_subgraphs*commBudget, p>=0, p<=1]
    problem = cp.Problem(cp.Maximize(obj_fn), constraint)
    problem.solve(solver='CVXOPT', kktsolver=cp.ROBUST_KKTSOLVER)

    # get solution
    tmp_p = p.value
    originActivationRatio = np.zeros((num_subgraphs))
    for i, pval in enumerate(tmp_p):
        originActivationRatio[i] = np.real(float(pval))

    return np.minimum(originActivationRatio, 1) 

def getAlpha(L_matrices, probabilities, num_nodes):
    num_subgraphs = len(L_matrices)
    
    # prepare matrices
    I = np.eye(num_nodes)
    J = np.ones((num_nodes, num_nodes))/num_nodes

    mean_L = np.zeros((num_nodes,num_nodes))
    var_L = np.zeros((num_nodes,num_nodes))
    for i in range(num_subgraphs):
        val = probabilities[i]
        mean_L += L_matrices[i]*val
        var_L += L_matrices[i]*(1-val)*val

    # SDP for mixing weight
    a = cp.Variable()
    b = cp.Variable()
    s = cp.Variable()
    obj_fn = s
    constraint = [(1-s)*I - 2*a*mean_L-J + b*(np.dot(mean_L,mean_L)+2*var_L) << 0, a>=0, s>=0, b>=0, cp.square(a) <= b]
    problem = cp.Problem(cp.Minimize(obj_fn), constraint)
    problem.solve(solver='CVXOPT', kktsolver=cp.ROBUST_KKTSOLVER)

    return  float(a.value)


In [2]:

G = pickle.load(open('two_clusters/2025-09-26/11-03-55/graph.pickle', 'rb'))
subgraphs = list()

# first try to get as many maximal matchings as possiblez
for i in range(80-1):
    M1 = nx.max_weight_matching(G)
    if nx.is_perfect_matching(G, M1):
        G.remove_edges_from(list(M1))
        subgraphs.append(list(M1))
    else:
        edge_list = list(G.edges)
        random.shuffle(edge_list)
        G.remove_edges_from(edge_list)
        G.add_edges_from(edge_list)

# use greedy algorithm to decompose the remaining part
rpart = decomposition(list(G.edges), 80)
for sgraph in rpart:
    subgraphs.append(sgraph)
subgraphs

[[(5, 0)],
 [(0, 1), (5, 6)],
 [(5, 9), (0, 4)],
 [(0, 2), (5, 7)],
 [(8, 5), (3, 0)]]

In [ ]:
edges = list(G.edges())
print(edges)
#probs = [data.get('probability_selection', 0) for _, _, data in graph.edges(data=True)]
#idxs = [idx for idx in range(len(edges)) if np.random.random() < probs[idx]]
max_degree = sorted(max(G.degree, key=lambda x: x[1]))
center_node_1, center_node_2 = max_degree
active_edges = []
if np.random.random() < 0.2:
    active_edges.append((center_node_1, center_node_2))
print(edges)
print(edges[~(edges == [center_node_1, center_node_2])])
# idxs = np.random.choice(len(edges), int(participation_rate * len(edges)), replace=False)
# active_edges = np.concatenate((active_edges, edges[idxs]))


[[0 1]
 [0 2]
 [0 5]
 [0 3]
 [0 4]
 [5 9]
 [5 8]
 [5 7]
 [5 6]]
[[0 1]
 [0 2]
 [0 5]
 [0 3]
 [0 4]
 [5 9]
 [5 8]
 [5 7]
 [5 6]]
[1 2 3 4 5 9 5 8 5 7 5 6]


In [7]:
size = 10
laplacians = graphToLaplacian(subgraphs, size)
probas = getProbability(laplacians, 2/5)
alpha = getAlpha(laplacians, probas, size)

In [4]:
W = list()
fraction_activated = dict()
for _ in range(100):
    L_k = np.sum([laplacians[i] for i in range(len(subgraphs)) if np.random.random() < probas[i]], axis=0)
    W = np.eye(size) - alpha * L_k 
    mask =  ~np.eye(W.shape[0], dtype=bool)
    non_zero_indices  = np.nonzero(W * mask)
    nodes_involved = set(non_zero_indices[0]) | set(non_zero_indices[1])
    for node in nodes_involved:
        if not node in fraction_activated:
            fraction_activated[node] = 1

In [5]:
fraction_activated

{np.int64(0): 1,
 np.int64(5): 1,
 np.int64(2): 1,
 np.int64(3): 1,
 np.int64(7): 1,
 np.int64(8): 1,
 np.int64(1): 1,
 np.int64(6): 1,
 np.int64(4): 1,
 np.int64(9): 1}

In [25]:
nodes_involved

{np.int64(0), np.int64(5)}